In [13]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import matplotlib.pyplot as plt
import numpy as np
import random
import cv2


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, silhouette_score
from sklearn.cluster import KMeans
import scipy as sp
from utils import *
from siamese import *
from visualize_attributes import *
from sklearn.manifold import TSNE
plt.ioff()

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
root_path = os.path.dirname(os.getcwd())
data_path = os.path.join(root_path, "data/unaugmented_multi_classed_grouped_data")
products = os.listdir(data_path)
visualize = True
generate_embeddings = False
generate_kmeans_labels = False

In [14]:
def kmeans_label(embeddings, save_path, save=False):
    # Perform k-means clutering on the embeddings, then save the labels as a new csv
    sil_scores = []
    # Find the optimal k using silhouette score
    for k in range(2, 9):
        kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto")
        kmeans.fit(embeddings)
        label_list = kmeans.labels_
        sil_scores.append(silhouette_score(embeddings, label_list, metric='euclidean'))

    optimal_k = list(range(2, 9))[np.argmax(sil_scores)]
    #print(optimal_k)

    # Redo k-means with the found optimal k
    kmeans = KMeans(n_clusters=optimal_k, random_state=0, n_init="auto")
    kmeans.fit(embeddings)
    label_list = kmeans.labels_

    # Save the label list as files, use these to calculate mahalonobis distance later
    if save:
        np.savetxt(save_path, label_list, delimiter=",")
    return kmeans.labels_


def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.T)
    inv_covmat = sp.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

In [23]:
embedding, product_embeddings, label_list
# Get all the cluster label available
all_labels = np.unique(label_list)
# Get the samples corresponding to the label
for label in all_labels:
    embedding_cluster = product_embeddings[np.where(label_list == label)[0]]
    # Calculate mahalanobis distance from the data point to the cluster
    distance = mahalanobis(embedding, embedding_cluster)
    print(distance)


LinAlgError: singular matrix

In [5]:
for backbone in ['resnet18', 'resnet34', 'resnet50', 'wide_resnet', 'vgg'][:1]:
    print('---------------------------------------------- ' + backbone + ' ----------------------------------------------')    
    for product in products[:1]:
        print('----------------------------------- ' + product + ' -----------------------------------')
        # Load corresponding Siamese weights
        model_path = os.path.join(root_path, 'models', product + '_siamese_' + backbone + '_subclass_sampling.pth')
        if not os.path.exists(model_path):
            print('No model for this backbone')
            continue
        print(model_path)
        if backbone in ['wide_resnet', 'vgg11']:
            device = 'cpu'

        # Load data dicts
        product_path = os.path.join(data_path, product)
        json_path = os.path.join(root_path, 'data/train_test_split', product)
        train_dict = json.load(open(os.path.join(json_path, product + '_train_dict.json'), 'r'))
        test_dict = json.load(open(os.path.join(json_path, product + '_test_dict.json'), 'r'))

        # Load data informations
        good_images_path = [image_path for image_path, label in train_dict.items()
                            if label == 1]


        # Load the embeddings and the list of id
        if generate_embeddings:
            product_embeddings, all_idx = map_good_train_samples_to_embeddings(json_path, train_dict, save=True)
        else:
            score_path = os.path.join(json_path, 'good_embeddings.csv')
            product_embeddings =  np.loadtxt(score_path, delimiter=",")
            all_idx = np.loadtxt(score_path[:-4] + '_id.csv', delimiter=",")
        product_embeddings_dict = dict(zip(all_idx, product_embeddings))

        if generate_kmeans_labels:
            save_path = os.path.join(json_path, 'labels.csv')
            label_list = kmeans_label(product_embeddings, save_path, save=True)
        else:
            label_list = np.loadtxt(os.path.join(json_path, 'labels.csv'), delimiter=",")

        # Prepare embedding model for test data
        embedding_model = torchvision.models.resnet50(weights='ResNet50_Weights.DEFAULT')
        modules=list(embedding_model.children())[:-1]
        embedding_model=nn.Sequential(*modules)
        for p in embedding_model.parameters():
            p.requires_grad = False
        
        # Load data to model
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Resize((1024,1024), antialias=True)])
        
        all_score = []
        i = 0
        # Embed the each test image and find its corresponding model images
        for image_path in tqdm(list(test_dict.keys())[:1]):
            image_dict = {image_path: test_dict[image_path]}

            # Load the test images to data loader
            embedding_dataset = EmbeddingDataset(image_dict, transform=transform)
            embedding_dataloader = DataLoader(embedding_dataset, batch_size=1, shuffle=True)
        
            # Calculate embedding:
            for data, _ in embedding_dataloader:
                data = data
                embedding = embedding_model(data)
            embedding = embedding.squeeze().detach().cpu().numpy()

            # Based on mahanalobis distance, firstly find the cluster where the embedding belongs
            

            # Find the candidate model images based on embedding distance:
            top_embeddings = dict(sorted(product_embeddings_dict.items(), 
                                    key=lambda x: distance(embedding, x[1]))) # Test Mahalanobis distance, check TSNE juan chua 
            


---------------------------------------------- resnet18 ----------------------------------------------
----------------------------------- bottle -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/bottle_siamese_resnet18_subclass_sampling.pth


100%|██████████| 1/1 [00:00<00:00,  1.27it/s]


In [4]:
for backbone in ['resnet18', 'resnet34', 'resnet50', 'wide_resnet', 'vgg'][1:]:
    print('---------------------------------------------- ' + backbone + ' ----------------------------------------------')    
    for product in products:
        print('----------------------------------- ' + product + ' -----------------------------------')
        # Load corresponding Siamese weights
        model_path = os.path.join(root_path, 'models', product + '_siamese_' + backbone + '_subclass_sampling.pth')
        if not os.path.exists(model_path):
            print('No model for this backbone')
            continue
        print(model_path)
        if backbone in ['wide_resnet', 'vgg11']:
            device = 'cpu'

        # Load data dicts
        product_path = os.path.join(data_path, product)
        json_path = os.path.join(root_path, 'data/train_test_split', product)
        train_dict = json.load(open(os.path.join(json_path, product + '_train_dict.json'), 'r'))
        test_dict = json.load(open(os.path.join(json_path, product + '_test_dict.json'), 'r'))

        # Load data informations
        good_images_path = [image_path for image_path, label in train_dict.items()
                            if label == 1]


        # Load the embeddings and the list of id
        if generate_embeddings:
            product_embeddings, all_idx = map_good_train_samples_to_embeddings(json_path, train_dict, product, save=True)
        else:
            score_path = os.path.join(json_path, 'good_embeddings.csv')
            product_embeddings =  np.loadtxt(score_path, delimiter=",")
            all_idx = np.loadtxt(score_path[:-4] + '_id.csv', delimiter=",")
        product_embeddings_dict = dict(zip(all_idx, product_embeddings))

   
        # Prepare embedding model for test data
        embedding_model = torchvision.models.resnet50(weights='ResNet50_Weights.DEFAULT')
        modules=list(embedding_model.children())[:-1]
        embedding_model=nn.Sequential(*modules)
        for p in embedding_model.parameters():
            p.requires_grad = False
        
        # Load data to model
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Resize((1024,1024), antialias=True)])
        
        all_score = []
        i = 0
        # Embed the each test image and find its corresponding model images
        for image_path in tqdm(list(test_dict.keys())):
            image_dict = {image_path: test_dict[image_path]}

            # Load the test images to data loader
            embedding_dataset = EmbeddingDataset(image_dict, transform=transform)
            embedding_dataloader = DataLoader(embedding_dataset, batch_size=1, shuffle=True)
        
            # Calculate embedding:
            for data, _ in embedding_dataloader:
                data = data
                embedding = embedding_model(data)
            embedding = embedding.squeeze().detach().cpu().numpy()

            # Find the candidate model images based on embedding distance:
            top_embeddings = dict(sorted(product_embeddings_dict.items(), 
                                    key=lambda x: distance(embedding, x[1]))) # Test Mahalanobis distance, check TSNE juan chua 
            
            # Return the dictionary where each key is the test image path and each value are the model images
            candidate_id_list = list(top_embeddings.keys())[:1]
            candidate_paths = [good_images_path[int(i)] for i in candidate_id_list]
            model_image_dict = dict(zip(candidate_paths, [1]*len(candidate_paths)))

            # Load the test image to the Siamese dataloader
            test_dataset = ProductDataset(image_dict, transform=transform)
            test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
            test_image, _, test_label, _ = next(iter(test_dataloader))
            test_label = test_label[0, 0]
            
            # Perform similarity score assessment with Siamese model
            all_distance = []
            # Load Siamese model
            n_classes = len(np.unique(list(train_dict.values())))
            model = SiameseNetwork(n_classes, backbone=backbone)
            transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Resize((1024,1024), antialias=True)
                                        ])
            
            model.load_state_dict(torch.load(model_path))
            model.eval()
            model.to(device)
            # Calculate distance from image to all model images
            for candidate_path in candidate_paths:
                model_image_dict = {candidate_path : 1}
                model_dataset = ProductDataset(model_image_dict, transform=transform)
                model_dataloader = DataLoader(model_dataset, batch_size=1, shuffle=True)
        
                model_image, _, model_label, _ = next(iter(model_dataloader))
                model_label = model_label[0, 0]
                # print(model_label, test_label)
        
                output_1, output_2 = model(model_image.to(device), test_image.to(device))
                euclidean_distance = F.pairwise_distance(output_1, output_2, keepdim=True)
        
                all_distance.append(euclidean_distance.item())

            # Save some pairs of images as visualization
            if i % 5 == 0:
                # Save inference figure
                model_image = np.array(model_image[0].permute(1, 2, 0)*255, dtype=np.uint8)
                test_image = np.array(test_image[0].permute(1, 2, 0)*255, dtype=np.uint8)
                label_1 = "good"
                label_2 = "defect" if test_label.numpy() == 0 else "good"
                label = "same class" if label_1 == label_2 else "different class"
                fig, ax = plt.subplots(1, 2, figsize=(10, 5))
                ax[0].imshow(model_image[:,:,::-1])
                ax[0].set_xlabel(label_1, weight = 'bold', fontsize = 20)
                #ax[0].axis('off')
                ax[1].imshow(test_image[:,:,::-1])
                ax[1].set_xlabel(label_2, weight = 'bold', fontsize = 20)
                #ax[1].axis('off')
                fig.suptitle('Dissimilarity score: ' + str(euclidean_distance.item()))
                figure_name = product + '_siamese_' + backbone + '_visualize_' + str(i) + '_embedded_scoring.png'
                figure_path = os.path.join(root_path, 
                        'Figure/visualize_inference/embedded_scoring', product)
                if not os.path.exists(figure_path):
                    os.mkdir(figure_path)
                figure_path = os.path.join(figure_path, figure_name)
                fig.savefig(figure_path)

                # Save scatter plot
                chosen_id = candidate_id_list[0]
                label_list = [1 if id != chosen_id else 2 for id in product_embeddings_dict.keys()]
                label_list.append(3)
                label_name_dict = {1 : 'good', 2 : 'chosen_good', 3 : 'test_image'}
                label_name = [label_name_dict[label] for label in label_list]
                #print(np.unique(label_dict))
                
                embedding_list = list(product_embeddings_dict.values())
                embedding_list.append(embedding)
                embedding_list = np.array(embedding_list)
                #print(embedding_list.shape)
                embedder = TSNE(2)
                embedding_list = embedder.fit_transform(embedding_list)
                #print(embedding_list.shape)
                
                fig, ax = plt.subplots()
                scatter = ax.scatter(embedding_list[:,0], embedding_list[:,1], c=label_list, label=label_name)
                ax.legend(handles=scatter.legend_elements()[0], labels=list(label_name_dict.values()))#, loc='upper right')
                fig.savefig(figure_path[:-4] + '_scatter_plot.png')
            
            # Calculate average distance between the test sample and all reference images
            avg_distance = np.mean(all_distance)
            # Keep track of all such distance
            all_score.append(np.array([1, int(test_label), 1 == int(test_label), avg_distance]))
            # print(avg_distance)
            i += 1
        # Save the results of all scores to file
        result = np.array(all_score)
        score_path = os.path.join(root_path, 'result/similarity_scores', product + '_siamese_' + backbone + '_mahalanobis_distance.csv')
        np.savetxt(score_path, result, delimiter=",")
        
        # Visualize similarity score histogram
        same_class = result[:,2]
        if len(np.unique(same_class)) != 2:
            print(np.unique(same_class))
        similarity_scores = result[:,3]
        
        # dissimillar_idx = np.where(same_class == 0.)
        # dissimillar_score = similarity_scores[dissimillar_idx]
        # simillar_idx = np.where(same_class == 1.)
        # simillar_score = similarity_scores[simillar_idx]
        # #print(dissimillar_idx, simillar_idx)
        # #print(dissimillar_score, simillar_score)
        # fig, ax = plt.subplots(1, 2, figsize=(16, 8))
        # ax[0].hist(simillar_score, edgecolor='black',)
        #            #bins=np.arange(simillar_score.min(), simillar_score.max(), 0.1))
        # ax[0].set_xlabel('Distance', fontsize = 10)
        # ax[0].set_ylabel('Number of instance', fontsize = 10)
        # ax[0].set_title(product + ': Similarity score distribution for images for GOOD class')
        # ax[1].hist(dissimillar_score, edgecolor='black',)
        #            #bins=np.arange(dissimillar_score.min(),dissimillar_score.max(), 0.1))
        # ax[1].set_xlabel('Distance', fontsize = 10)
        # ax[1].set_ylabel('Number of instance', fontsize = 10)
        # ax[1].set_title(product + ': Similarity score distribution for images for DEFECT class')
        # fig.show()

        # figure_name = product + '_siamese_' + backbone + '_similarity_score_distribution' + '_mahalanobis_embedded_scoring.png'
        # figure_path = os.path.join(root_path, 
        #         'Figure/embedded_similarity_score_histogram/' + figure_name)
        # fig.savefig(figure_path)
            

---------------------------------------------- resnet18 ----------------------------------------------
----------------------------------- bottle -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/bottle_siamese_resnet18_subclass_sampling.pth


 85%|████████▍ | 50/59 [00:54<00:08,  1.08it/s]/tmp/ipykernel_2656172/595678416.py:116: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1, 2, figsize=(10, 5))
100%|██████████| 59/59 [01:04<00:00,  1.09s/it]
/tmp/ipykernel_2656172/595678416.py:184: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


----------------------------------- cable -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/cable_siamese_resnet18_subclass_sampling.pth


100%|██████████| 77/77 [01:27<00:00,  1.13s/it]
/tmp/ipykernel_2656172/595678416.py:184: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


----------------------------------- capsule -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/capsule_siamese_resnet18_subclass_sampling.pth


...

----------------------------------- zipper -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/zipper_siamese_resnet34_subclass_sampling.pth


100%|██████████| 82/82 [01:30<00:00,  1.11s/it]
/tmp/ipykernel_2656172/595678416.py:184: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


---------------------------------------------- resnet50 ----------------------------------------------
----------------------------------- bottle -----------------------------------
No model for this backbone
----------------------------------- cable -----------------------------------
No model for this backbone
----------------------------------- capsule -----------------------------------
No model for this backbone
----------------------------------- carpet -----------------------------------
No model for this backbone
----------------------------------- grid -----------------------------------
No model for this backbone
----------------------------------- hazelnut -----------------------------------
No model for this backbone
----------------------------------- leather -----------------------------------
No model for this backbone
----------------------------------- metal_nut -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_asses

  1%|▏         | 1/68 [00:05<06:15,  5.61s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 9.75 GiB total capacity; 6.17 GiB already allocated; 85.25 MiB free; 6.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF